In [28]:
# -*- coding: utf-8 -*-
"""submit.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1INEp68lK0IT1HEKflwAjaB-nflNYeF3a
"""

import numpy as np
import sklearn
from scipy.linalg import khatri_rao
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

def my_fit( X_train, y_train ):
    X = my_map2(X_train)
    log_reg = LinearSVC()
    log_reg.fit(X, y_train)
    print(log_reg.score(X, y_train))
    w = log_reg.coef_
    b = log_reg.intercept_
    w = w.flatten()
    return w, b


def my_map(X_train):
  D = 1- 2*X_train
  X = np.zeros((len(X_train),32))
  mapped_X = []
  for k in range(len(X_train)):
      arr = X_train[k]
      i, j = np.tril_indices(len(arr), k=-1)
      diag_ind = np.diag_indices(len(arr))

      products = arr[i] * arr[j]
      products_diag = arr[diag_ind[0]] * arr[diag_ind[1]]

      all_products = np.concatenate((products,products_diag))
      mapped_X.append(all_products)
  #   for j in range(32):
  #     X[i][j] = np.prod(D[i][j:])
  # X_matrix = []
  # for i in range(len(X)):
  #   X_matrix_row=[]
  #   for j in range(32):
  #     X_matrix_row.append(X[i][j]**2)
  #     for k in range(j+1,32):
  #       X_matrix_row.append(X[i][j]*X[i][k])
  #   X_matrix.append(X_matrix_row)

  # X_matrix = np.array(X_matrix)
  return np.array(mapped_X)

In [63]:
import numpy as np
import sklearn
from scipy.linalg import khatri_rao
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
def my_fit( X_train, y_train ):
    X = my_map(X_train)
    log_reg = LinearSVC(loss='squared_hinge', penalty='l2', C=40)
    log_reg.fit(X, y_train)
    print(log_reg.score(X, y_train))
    w = log_reg.coef_
    b = log_reg.intercept_
    w = w.flatten()
    return w, b


def my_map(X_train):
  D = 1- 2*X_train
  X = np.zeros((len(X_train),32))
  for i in range(len(X_train)):
      for j in range(32):
          X[i][j] = np.prod(D[i][j:])
  mapped_X = []
  for k in range(len(X)):
      arr = X[k]
      i, j = np.tril_indices(len(arr), k=-1)

      products = arr[i] * arr[j]
      to_append = np.concatenate((arr, products))
      mapped_X.append(to_append)
  return np.array(mapped_X)

In [64]:
Z_trn = np.loadtxt( "train.dat" )
Z_tst = np.loadtxt( "test.dat" )
X_train, Y_train = Z_trn[:, :-1], Z_trn[:,-1]
X_test, Y_test = Z_tst[:, :-1], Z_tst[:,-1]
w,b = my_fit(X_train, Y_train)
print(b)

0.995075
[-11.55674934]


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [65]:
test = my_map(X_test)
w.shape, X_test.shape, test.shape

((528,), (10000, 32), (10000, 528))

In [66]:
scores = np.dot(test,w) + b

In [67]:
scores.shape

(10000,)

In [68]:
pred = np.zeros_like(scores)
pred[scores > 0] = 1

In [69]:
pred.shape

(10000,)

In [70]:
accuracy_score(Y_test, pred)

0.9904